# RNNs exercises

**1. The Lorenz ODE system as defined as**

$$
\begin{align}
\frac{dx}{dt} &= \sigma(y - x),\\
\frac{dy}{dt} &= x(r - z) - y,\\
\frac{dz}{dt} &= xy - bz,
\end{align}\tag{1}
$$
**where $\sigma$, $r$ and $b$ are parameters. Define a function that takes a numpy array of shape `(batch_size, 3)` as input, along with parameters `sigma`, `r` and `b`, and returns a numpy array of shape `(batch_size, 3)` where the entries are $dx/dt$, $dy/dt$ and $dz/dt$ for each element in the batch.**

In [ ]:
import numpy as np

def lorenzf(inputs, sigma=10, r=28, b=8/3):
    x, y, z = inputs[:, 0], inputs[:, 1], inputs[:, 2]
    dxdt = sigma * (y - x)
    dydt = (x * (r - z)) - y
    dzdt = (x * y) - (b * z)
    outputs = np.vstack((dxdt, dydt, dzdt)).T
    return outputs

**2. Write a function that takes an initial condition (a numpy array of shape `(batch_size, 3)`) and numerically computes a solution to the Lorenz equations. Your function should also take `n_steps` and `step_size` as arguments, and use the Euler scheme to numerically solve the ODE:**

$$
(x_{n+1}, y_{n+1}, z_{n+1}) = (x_{n+1}, y_{n+1}, z_{n+1}) + \eta f(x_n, y_n, z_n),
$$
**where $f:\mathbb{R}^3\mapsto\mathbb{R}^3$ is the right hand side of (1), and $\eta$ is the step size. Your function should return a numpy array of shape `(batch_size, n_steps + 1, 3)`.**

**Demonstrate your function by computing a batch of two orbits and plotting them in the $(y, z)$-plane. Use a step size of 0.01.**

In [ ]:
def euler_step(init_, step_size=0.01, sigma=10, r=28, b=8/3):
    return init_ + (step_size * lorenzf(init_, sigma=sigma, r=r, b=b))

def euler_steps(init, n_steps, step_size=0.01, sigma=10, r=28, b=8/3):
    orbit = init[:, np.newaxis, :]
    for _ in range(n_steps):
        orbit_next = euler_step(orbit[:, -1, :], step_size=step_size, sigma=sigma, r=r, b=b)[:, np.newaxis, :]
        orbit = np.concatenate((orbit, orbit_next), axis=-2)
    return orbit

In [ ]:
batch_size = 2

init = np.random.uniform(low=-15, high=15, size=batch_size * 3).reshape((batch_size, 3))
orbit = euler_steps(init, n_steps=5000, step_size=0.01)
orbit.shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))

plt.subplot(1, 2, 1)
plt.plot(orbit[0, :, 1], orbit[0, :, 2])
plt.xlabel(r"$y$")
plt.ylabel(r"$z$")
plt.subplot(1, 2, 2)
plt.plot(orbit[1, :, 1], orbit[1, :, 2])
plt.xlabel(r"$y$")
plt.ylabel(r"$z$")
plt.show()

**3. Generate a dataset of numerically computed orbits of the Lorenz system, using initial conditions with each $x,y,x$ sampled uniformly from -15 to 15. Each orbit should consist of 5000 time steps, and there should be 400 orbits in the training set and 100 orbits in the test set. The data arrays should have shape `(N, 5000, 3)`. Call these arrays `y_train` and `y_test` respectively.**

In [ ]:
y_train = []

for _ in range(4):
    batch_size = 100
    init = np.random.uniform(low=-15, high=15, size=batch_size * 3).reshape((batch_size, 3))
    orbit = euler_steps(init, n_steps=4999, step_size=0.01)
    y_train.append(orbit)
y_train = np.concatenate(y_train, axis=0)
print(y_train.shape)

batch_size = 100
init = np.random.uniform(low=-15, high=15, size=batch_size * 3).reshape((batch_size, 3))
y_test = euler_steps(init, n_steps=4999, step_size=0.01)
print(y_test.shape)

**4. Now create noisy versions of these orbits, by adding i.i.d. Gaussian noise to each point in each orbit. The Gaussian noise should have mean zero and standard deviation 0.5. The 'noisy' arrays should have shape `(N, 5000, 3)`. Call these arrays `x_train` and `x_test`.**

**Plot a sample orbit from the training set in the $(y, z)$-plane, showing the 'true' and 'noisy' versions of the orbit.**

In [ ]:
std = 0.5

x_train = y_train + np.random.randn(*y_train.shape) * std
x_test = y_test + np.random.randn(*y_test.shape) * std

In [ ]:
i = np.random.choice(x_train.shape[0])
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(y_train[i, :, 1], y_train[i, :, 2])
ax1.set_xlabel(r"$y$")
ax1.set_ylabel(r"$z$")
ax1.set_title("True orbit")
ax2.plot(x_train[i, :, 1], x_train[i, :, 2])
ax2.set_xlabel(r"$y$")
ax2.set_ylabel(r"$z$")
ax2.set_title("Noisy orbit")
plt.show()

**5. Define and train an RNN model that takes a noisy orbit as input, and predicts the true/de-noised orbit. You are free to choose the design of your network, as well as the training algorithm. Print the final test RMSE obtained by your model.**

**You might find that it takes a while to train a good model. It may help to speed up training if you normalise the data somehow.**

In [ ]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dense

model = Sequential([
    Input(shape=[None, 3]),
    LSTM(32, return_sequences=True),
    LSTM(16, return_sequences=True),
    Dense(3)
])
model.summary()

In [ ]:
# Normalise data by subtracting mean and dividing by STD

mean = np.mean(y_train.reshape([-1, 3]), axis=0)
std = np.std(y_train.reshape([-1, 3]), axis=0)

x_train_norm = (x_train - mean) / std
y_train_norm = (y_train - mean) / std
mean.shape, std.shape

In [ ]:
x_test_norm = (x_test - mean) / std
y_test_norm = (y_test - mean) / std

In [ ]:
# Compute the following RMSE as a baseline

np.sqrt(np.mean(np.square(x_train_norm - y_train_norm)))

In [ ]:
from keras.callbacks import EarlyStopping

model.compile(loss='mse', optimizer='adam', metrics=['mae', 'root_mean_squared_error'])
earlystopping = EarlyStopping(patience=5)
history = model.fit(x_train_norm, y_train_norm, epochs=100, batch_size=20, validation_data=(x_test_norm, y_test_norm), 
                    callbacks=[earlystopping])

In [ ]:
model.evaluate(x_train_norm, y_train_norm)

In [ ]:
model.evaluate(x_test_norm, y_test_norm)

**6. Plot an example orbit from the test set in the $(y, z)$-plane, including the true version, noisy version, and model prediction.**

In [ ]:
i = np.random.choice(x_test.shape[0])

display_nsteps = 1000

fig, axes = plt.subplots(1, 3, figsize=(14, 4))
ax1, ax2, ax3 = axes.flatten()
ax1.plot(y_test[i, :display_nsteps, 1], y_test[i, :display_nsteps, 2], alpha=0.8)
ax1.set_xlabel(r"$y$")
ax1.set_ylabel(r"$z$")
ax1.set_title("True orbit")
ax2.plot(x_test[i, :display_nsteps, 1], x_test[i, :display_nsteps, 2], alpha=0.8)
ax2.set_xlabel(r"$y$")
ax2.set_ylabel(r"$z$")
ax2.set_title("Noisy orbit")

pred = model(x_test_norm[i:i+1]).numpy()
pred = (pred * std) + mean

ax3.plot(pred[0, :display_nsteps, 1], pred[0, :display_nsteps, 2], alpha=0.8)
ax3.set_xlabel(r"$y$")
ax3.set_ylabel(r"$z$")
ax3.set_title("Model prediction")
plt.show()